Исходник датасета: https://www.kaggle.com/aerodinamicc/ecommerce-website-funnel-analysis


In [ ]:
# Загружаем файлы, который прислал Александр

In [ ]:
# импортируем нужные библиотеки
import pandas as pd
from plotly import graph_objects as go

In [ ]:
# Загружаем в Python с помощью Pandas все датасеты
user_table = pd.read_csv('user_table.csv')

# Просматриваем первые 5 строк датасета
user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [ ]:
user_table.tail() # Просматриваем последние 5 строк датасета

,user_id,date,device,sex
27931,22996,2015-01-27,Mobile,Male
27932,318106,2015-04-07,Mobile,Male
27933,367806,2015-02-01,Desktop,Female
27934,739265,2015-04-21,Desktop,Female
27935,368767,2015-02-13,Mobile,Female


In [ ]:
# Просматриваем кол-во строк и столбцов каждого датасета
user_table.shape

(27936, 4)

In [ ]:
# Загружаем в Python 
home_page = pd.read_csv('home_page_table.csv')
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [ ]:
# Просматриваем кол-во строк и столбцов каждого датасета
home_page.shape

(90400, 2)

In [ ]:
sum(home_page.duplicated())

0

In [ ]:
search_page = pd.read_csv('search_page_table.csv')
search_page.head()

,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page
3,780347,search_page
4,383739,search_page


In [ ]:
search_page.shape

(45200, 2)

In [ ]:
payment_page = pd.read_csv('payment_page_table.csv')
payment_page.head()

,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page
3,901286,payment_page
4,195052,payment_page


In [ ]:
payment_page.shape

(6030, 2)

In [ ]:
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')
payment_confirmation.head()

,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page
3,538348,payment_confirmation_page
4,841681,payment_confirmation_page


In [ ]:
payment_confirmation.shape

(452, 2)

In [ ]:
# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [ ]:
# Склеиваем последовательно все датасеты с помощью метода merge по идентификатору user_id
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [ ]:
df.shape

(27936, 8)

In [ ]:
# Второй способ: склеиваем все датасеты сразу
from functools import reduce

data_frames = [user_table, home_page, search_page, payment_page, payment_confirmation]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on='user_id', how='left'), data_frames)

df_merged.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [ ]:
df_merged.shape

(27936, 8)

In [ ]:
# Для удоства последующих расчетов, перекодируем названия событий в 0/1 с помощью лямбда-функции
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [ ]:
# Агрегируем данные: подсчитываем количество пользователей по разным типам устройств
df['device'].value_counts()

Desktop    18510
Mobile      9426
Name: device, dtype: int64

In [ ]:
# Агрегируем данные: подсчитываем общее количество пользователей и сколько из них подтвердили покупку
df['payment_confirmation'].agg({'users':'count', 'payers':'sum'})

users     27936
payers      145
Name: payment_confirmation, dtype: int64

In [ ]:
# Агрегируем данные: подсчитываем сколько пользователей совершило каждое из 4х событий на сайте
summary = df[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().reset_index()
summary.head()

,index,0
0,home_page,27936
1,search_page,14031
2,payment_page,1928
3,payment_confirmation,145


In [ ]:
# Для удобства переименуем столбцы
summary= summary.rename(columns = {'index':'stage', 0:'number'})
summary.head()

,stage,number
0,home_page,27936
1,search_page,14031
2,payment_page,1928
3,payment_confirmation,145


In [ ]:
# Импортируем графическую библиотеку Plotly (https://plotly.com/python/funnel-charts/)
#from plotly import graph_objects as go

# Настраиваем скрипт этой библиотеки для построения воронки конверсий нашими параметрами
fig = go.Figure(go.Funnel(
    y = summary['stage'],
    x = summary['number'],
    textinfo = "value+percent previous"))

fig.update_layout(
    title = 'Воронка конверсии Интернет-магазина',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()

In [ ]:
# Всетаки лучше сегментировать воронку конверсий в разрезе характеристик пользователей
# Подготовим группировку данных по совершению событий на сайте в разрезе типов устройств, используя метод groupby
# Метод reset_index() используется для того, чтобы превратить индекс, по которому группируем данные (device), в отдельную колонку датафрейма
dfg = df.groupby('device')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('device').unstack('device').reset_index()

# Уберем ненужный заголовок верхнего уровня
dfg.columns.name = None

# Переименуем стандартные после процедуры заголовки
dfg = dfg.rename(columns = {'index':'Page'})

# Посмотрим, что получилось
dfg.head()

,Page,Desktop,Mobile
0,home_page,18510,9426
1,search_page,9252,4779
2,payment_page,972,956
3,payment_confirmation,43,102


In [ ]:
# Строим более продвинутый вариант воронки конверсий на сайте с сегментацией по типам устройств


# Настраиваем скрипт библиотеки Plotly нашими параметрами
fig = go.Figure()

# Часть графика для типа устройств Dekstop
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Desktop'],
    name = 'Desktop',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

# Часть графика для типа устройств Mobile
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Mobile'],
    name = 'Mobile',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии по типам устройств',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()

А теперь потренируемся в создании сводных таблиц методами groupby, pivot_table, crosstab для ответов на поставленные вопросы:

1) Аудитория какого пола и на каких типах устройств больше всего у интернет-магазина?

In [ ]:
# Используем метод pivot_table
pd.pivot_table(df, index = 'sex', columns = 'device', values = 'home_page', aggfunc = 'sum', margins=True)

device,Desktop,Mobile,All
sex,,,
Female,9088,4643,13731
Male,9422,4783,14205
All,18510,9426,27936


In [ ]:
# Используем функцию crosstab
# (по умолчанию рассчитывает частоты, поэтому valus = df['user_id'] и aggfunc = 'count' можно не указывать)
pd.crosstab(index = df['sex'], columns = df['device'], margins = True)

device,Desktop,Mobile,All
sex,,,
Female,9088,4643,13731
Male,9422,4783,14205
All,18510,9426,27936


In [ ]:
# Используем метод groupby
df.groupby(['sex', 'device'])['user_id'].count()

sex     device 
Female  Desktop    9088
        Mobile     4643
Male    Desktop    9422
        Mobile     4783
Name: user_id, dtype: int64

In [ ]:
# Немного не то представление получилось, да? Попробуем добавить метод unstack()
df.groupby(['sex', 'device'])['user_id'].count().unstack('device')

device,Desktop,Mobile
sex,,
Female,9088,4643
Male,9422,4783


2) Кто чаще всего останавливается только на поиске товаров в интернет-магазине - мужчины или женщины?

In [ ]:
# С этой задачей лучше справится groupby
(df.groupby('sex')['search_page'].sum() - df.groupby('sex')['payment_page'].sum()) / df.groupby('sex')['search_page'].sum()

sex
Female    0.856792
Male      0.868232
dtype: float64

3) Кто чаще подтверждает платеж и совершает окончательную покупку?

In [ ]:
# Используем функцию crosstab и ее параметр normalize для получения относительных частот 
# (по умолчанию рассчитывает частоты, поэтому valus = df['user_id'] и aggfunc = 'count' можно не указывать)
pd.crosstab(index = df['sex'], columns = df['payment_confirmation'], normalize='index')

payment_confirmation,0,1
sex,,
Female,0.994101,0.005899
Male,0.995495,0.004505


In [ ]:
# Используем метод pivot_table (по умолчанию рассчитывает средние значения, поэтому  aggfunc = 'mean' можно не указывать)
df.pivot_table(index = ['sex'], values = 'payment_confirmation')

,payment_confirmation
sex,
Female,0.005899
Male,0.004505


In [ ]:
# Используем метод groupby
df.groupby('sex')['payment_confirmation'].mean()

sex
Female    0.005899
Male      0.004505
Name: payment_confirmation, dtype: float64

4) И на каких устройствах?

In [ ]:
# Используем функцию crosstab и ее параметр normalize для получения относительных частот 
pd.crosstab(index = [df['sex'], df['device']], columns = df['payment_confirmation'], normalize='index')

payment_confirmation         0         1
sex    device                           
Female Desktop        0.997579  0.002421
       Mobile         0.987293  0.012707
Male   Desktop        0.997771  0.002229
       Mobile         0.991010  0.008990

In [ ]:
# Используем метод pivot_table
df.pivot_table(index = ['sex','device'], values = 'payment_confirmation')

payment_confirmation
sex    device                       
Female Desktop              0.002421
       Mobile               0.012707
Male   Desktop              0.002229
       Mobile               0.008990

In [ ]:
# Используем метод groupby
df.groupby(['sex', 'device'])['payment_confirmation'].mean()

sex     device 
Female  Desktop    0.002421
        Mobile     0.012707
Male    Desktop    0.002229
        Mobile     0.008990
Name: payment_confirmation, dtype: float64

### ***Спасибо за внимание!***